In [ ]:
pip install torch

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
pip install transformers

In [ ]:
import torch
import transformers

In [ ]:
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
import pickle
import json
import os

In [ ]:
# Placeholder data
log_errors = ["ERROR", "CRITICAL", "WARNING"]
error_messages = ["Database Connection Failed", "Out of Memory Error", "Disk Space Low"]
possible_causes = [
    ["Network issues", "Database server down", "Incorrect credentials"],
    ["Memory leak", "Increased workload", "Inefficient code"],
    ["Accumulation of log files", "Continuous data growth", "Uncontrolled file uploads"]
]
possible_fixes = [
    ["Check network connectivity", "Verify database server status", "Review database credentials"],
    ["Memory profiling", "Optimize code", "Scaling"],
    ["Log rotation", "Regular cleanup tasks", "Monitor and manage uploads"]
]

In [ ]:
# Placeholder data
num_log_error_classes = len(log_errors)
num_error_message_classes = len(error_messages)
num_possible_causes_classes = len(possible_causes)
num_possible_fixes_classes = len(possible_fixes)

total_number_of_classes = num_log_error_classes + num_error_message_classes + num_possible_causes_classes + num_possible_fixes_classes

model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=total_number_of_classes)

max_length = 128


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Initialize BERT model for sequence classification
num_log_error_classes = len(log_errors)
num_error_message_classes = len(error_messages)
num_possible_causes_classes = len(possible_causes)
num_possible_fixes_classes = len(possible_fixes)

total_number_of_classes = num_log_error_classes + num_error_message_classes + num_possible_causes_classes + num_possible_fixes_classes

model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=total_number_of_classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load the trained model's state_dict from the pickle file
model_state_dict_path = '/content/drive/MyDrive/SentenceLevelModelFinal/SentenceLevelModelV8_state_dict.pkl'# why creating

In [ ]:
model.load_state_dict(torch.load(model_state_dict_path))

<All keys matched successfully>

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Set model to evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
# Function for predicting log error
def predict_log_error(input_text):
    print("\n--- Tokenization ---")
    encoding = tokenizer(input_text, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    print("Tokenization completed.")

    print("\n--- Tokens ---")
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())
    print("Tokens:", tokens)

    print("\n--- Positional Embeddings ---")
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)

    print("\n--- Padding & Truncation ---")
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)

    print("\n--- Data Vectors ---")
    print("Data Vectors - Tokenized input IDs:", input_ids)
    print("Data Vectors - Attention mask:", attention_mask)

    print("\n----Prediction for Log Error----")
    outputs = model(input_ids, attention_mask=attention_mask)
    log_error_prediction = torch.argmax(outputs.logits, dim=1).item()
    log_error_label = log_errors[log_error_prediction]
    print("Predicted Log Error Label:", log_error_label)
    print("Predicted Log Error Token:", tokens[torch.argmax(outputs.logits, dim=1).item()])
    print("Prediction for Log_Error completed.")

    return log_error_label

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# Function for predicting possible cause
def predict_possible_cause(input_text):
    encoding = tokenizer(input_text, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    print("--- Tokenization ---")
    print("Input Text:", input_text)
    print("Tokenization step:")
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    outputs = model(input_ids, attention_mask=attention_mask)
    possible_causes_logits = outputs.logits[:, num_log_error_classes:num_log_error_classes+num_possible_causes_classes]
    possible_causes_prediction = torch.argmax(possible_causes_logits, dim=1).item()
    possible_causes_label = possible_causes[possible_causes_prediction]
    print("------ Prediction for Possible cause ------")
    print("Predicted Possible Cause:", possible_causes_label)
    print("Token IDs for Predicted Possible Cause:")
    for cause in possible_causes_label:
        encoding = tokenizer(cause, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].to(device)
        print(cause, ":", input_ids)
    return possible_causes_label

# Function for predicting possible fix
def predict_possible_fix(input_text):
    encoding = tokenizer(input_text, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    print("--- Tokenization ---")
    print("Input Text:", input_text)
    print("Tokenization step:")
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    outputs = model(input_ids, attention_mask=attention_mask)
    possible_fixes_logits = outputs.logits[:, num_log_error_classes + num_possible_causes_classes:]
    possible_fixes_prediction = torch.argmax(possible_fixes_logits, dim=1).item()
    possible_fixes_label = possible_fixes[possible_fixes_prediction]
    print("------ Prediction for Possible Fix ------")
    print("Predicted Possible Fix:", possible_fixes_label)
    print("Token IDs for Predicted Possible Fix:")
    for fix in possible_fixes_label:
        encoding = tokenizer(fix, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].to(device)
        print(fix, ":", input_ids)
    return possible_fixes_label

In [ ]:
# Loop for predicting errors until "exit"
while True:
    user_input = input("Enter the error message (type 'exit' to stop): ")

    if user_input.lower() == 'exit':
        print("Exiting the program.")
        break

    log_error_result = predict_log_error(user_input)
    possible_cause_result = predict_possible_cause(user_input)
    possible_fix_result = predict_possible_fix(user_input)

Enter the error message (type 'exit' to stop): Database connection failed

--- Tokenization ---
Tokenization completed.

--- Tokens ---
Tokens: ['[CLS]', 'database', 'connection', 'failed', '[SEP]']

--- Positional Embeddings ---
Input IDs shape: torch.Size([1, 5])
Attention Mask shape: torch.Size([1, 5])

--- Padding & Truncation ---
Input IDs: tensor([[ 101, 7809, 4434, 3478,  102]], device='cuda:0')
Attention Mask: tensor([[1, 1, 1, 1, 1]], device='cuda:0')

--- Data Vectors ---
Data Vectors - Tokenized input IDs: tensor([[ 101, 7809, 4434, 3478,  102]], device='cuda:0')
Data Vectors - Attention mask: tensor([[1, 1, 1, 1, 1]], device='cuda:0')

----Prediction for Log Error----
Predicted Log Error Label: ERROR
Predicted Log Error Token: [CLS]
Prediction for Log_Error completed.
--- Tokenization ---
Input Text: Database connection failed
Tokenization step:
Input IDs: tensor([[ 101, 7809, 4434, 3478,  102]], device='cuda:0')
Attention Mask: tensor([[1, 1, 1, 1, 1]], device='cuda:0')
--